In [8]:
import numpy
import datetime

class Investment(object):
    def __init__(self, price, ticker):
        self.ticker = str(ticker)
        self.price = float(price)
    
    def __str__(self):
        return '%s: $%.2f per share' % (self.ticker, self.price)
    
    def __repr__(self):
        return self.__str__()
    
class Stock(Investment):
    def __init__(self, price, ticker):
        if isinstance(price, float):
            print """
    Stock prices must be integer values.
    Input rounded down to $%d.
            """% (int(price))
        Investment.__init__(self, int(price), ticker)
        
class MutualFund(Investment):
    def __init__(self, ticker, price = 1):
        Investment.__init__(self, 1, ticker)

class Bond(Investment):
    def __init__(self, price, ticker):
        Investment.__init__(self, price, ticker)
        
class Portfolio(object):
    def __init__(self, cash = 0):
        self.cash = float(cash)
        self.stock = 'None'
        self.mf = 'None'
        self.bond = 'None'
        self.transactions = []
    
    def __str__(self):
        return """
    As of %s
    -----------------------
    Cash:    %.2f
    Stock:   %s
    M.funds: %s
    Bonds:   %s
        """ % (self.stampTime(), 
               self.cash, 
               self.sortInvestments(self.stock), 
               self.sortInvestments(self.mf), 
               self.sortInvestments(self.bond))
    
    def sortInvestments(self, investment):
        if investment == 'None':
            return investment
        else:
            show_investment = sorted(['%s --- %.2f shares' % (i, investment[i]) for i in investment])
            return '\n\t     '.join(show_investment)
    
    def __repr__(self):
        return self.__str__()
    
    def stampTime(self):
        return datetime.datetime.now().strftime("%I:%M%p on %m/%d/%Y")
        
    def addCash(self, amount, is_add = True):
        self.cash += float(amount)
        if is_add:
            self.transactions.append('%s | Added cash: $%.2f' % (self.stampTime(), amount))
        
    def withdrawCash(self, amount, is_cash = True, shares = 0, investment = Stock(1, 'Empty')):
        if amount > self.cash:
            line1 = 'Insufficient funds.'
        else:
            self.addCash(-amount, is_add = False)
            if is_cash:
                line1 = 'Withdrew $%.2f from your portfolio.' % (amount)
                self.transactions.append('%s | Withdrew cash: $%.2f' % (self.stampTime(), amount))
            else: 
                line1 = 'You bought %.2f shares of %s for $%.2f.' % (shares, investment.ticker, amount)
                self.transactions.append('%s | Bought shares: %.2f (%s) @ $%.2f/s ($%.2f total)' % (self.stampTime(), 
                                                                                                    shares, 
                                                                                                    investment.ticker, 
                                                                                                    investment.price, 
                                                                                                    amount))
        print """
    %s
    You have $%.2f remaining in your portfolio.
        """ % (line1, self.cash)
        
    def buyInvestment(self, shares, investment, my_shares):
        cost = shares * investment.price
        self.withdrawCash(cost, False, shares, investment)
        if my_shares == 'None':
            my_shares = {investment.ticker : shares}
        elif investment.ticker in my_shares.keys():
            my_shares[investment.ticker] += shares
        else:
            my_shares[investment.ticker] = shares
        return my_shares
        
    def buyStock(self, shares, investment):
        self.stock = self.buyInvestment(shares, investment, self.stock)
    
    def buyMutualFund(self, shares, investment):
        self.mf = self.buyInvestment(shares, investment, self.mf)
    
    def buyBond(self, shares, investment):
        self.bond = self.buyInvestment(shares, investment, self.bond)
     
    def sellInvestment(self, shares, investment, my_shares, sale_price):
        if investment.ticker not in my_shares:
            print 'You do not own any shares of that investment.'
            return my_shares
        elif shares > my_shares[investment.ticker]:
            print """
    Cannot sell %.2f shares of %s. 
    You only own %.2f shares.
            """ % (shares, investment.ticker, my_shares[investment.ticker])
            return my_shares
        else:
            earnings = sale_price * investment.price * shares
            profit = earnings - (shares * investment.price)
            self.cash += earnings
            if shares == my_shares[investment.ticker]:
                del my_shares[investment.ticker]
            else:
                my_shares[investment.ticker] -= shares
            print """
    Sold %.2f shares of %s at $%.2f per share.
    You have %.2f shares remaining.
    The sale was worth $%.2f, for a profit of $%.2f.
    You now have $%.2f in your portfolio.
            """ % (shares, 
                   investment.ticker, 
                   sale_price * investment.price, 
                   my_shares[investment.ticker], 
                   earnings, profit, 
                   self.cash)
            self.transactions.append('%s | Sold shares: %.2f (%s) for $%.2f/s ($%.2f total)' % (self.stampTime(), 
                                                                                                shares, 
                                                                                                investment.ticker, 
                                                                                                sale_price * investment.price, 
                                                                                                sale_price * investment.price * shares))
            investment.price = sale_price
            return my_shares
    
    def sellStock(self, shares, investment):
        self.stock = self.sellInvestment(shares, investment, self.stock, numpy.random.uniform(0.5, 1.5))

    def sellMutualFund(self, shares, investment):
        self.mf = self.sellInvestment(shares, investment, self.mf, numpy.random.uniform(0.9, 1.2))

    def sellBond(self, shares, investment):
        self.bond = self.sellInvestment(shares, investment, self.bond, 1.02)
    
    def history(self, from_start = True):
        if from_start:
            print '\n'.join(self.transactions)
        else:
            print '\n'.join(reversed(self.transactions))

In [9]:
portfolio = Portfolio()

In [10]:
portfolio.addCash(300.50)

In [11]:
portfolio


    As of 03:39PM on 08/08/2018
    -----------------------
    Cash:    300.50
    Stock:   None
    M.funds: None
    Bonds:   None
        

In [12]:
s = Stock(20, 'HFH')

In [13]:
s

HFH: $20.00 per share

In [14]:
s2 = Stock(20.3, 'HFH2')


    Stock prices must be integer values.
    Input rounded down to $20.
            


In [15]:
portfolio.buyStock(5, s)


    You bought 5.00 shares of HFH for $100.00.
    You have $200.50 remaining in your portfolio.
        


In [16]:
mf1 = MutualFund('BRT')

In [17]:
mf2 = MutualFund('GHT')

In [18]:
portfolio.buyMutualFund(10.3, mf1)


    You bought 10.30 shares of BRT for $10.30.
    You have $190.20 remaining in your portfolio.
        


In [19]:
portfolio.buyMutualFund(2, mf2)


    You bought 2.00 shares of GHT for $2.00.
    You have $188.20 remaining in your portfolio.
        


In [20]:
print portfolio


    As of 03:39PM on 08/08/2018
    -----------------------
    Cash:    188.20
    Stock:   HFH --- 5.00 shares
    M.funds: BRT --- 10.30 shares
	     GHT --- 2.00 shares
    Bonds:   None
        


In [21]:
portfolio.sellMutualFund(3, mf1)


    Sold 3.00 shares of BRT at $1.14 per share.
    You have 7.30 shares remaining.
    The sale was worth $3.42, for a profit of $0.42.
    You now have $191.62 in your portfolio.
            


In [22]:
portfolio.sellStock(1, s)


    Sold 1.00 shares of HFH at $24.83 per share.
    You have 4.00 shares remaining.
    The sale was worth $24.83, for a profit of $4.83.
    You now have $216.45 in your portfolio.
            


In [23]:
portfolio.sellStock(1, mf1)

You do not own any shares of that investment.


In [24]:
portfolio.sellStock(100, s)


    Cannot sell 100.00 shares of HFH. 
    You only own 4.00 shares.
            


In [25]:
portfolio.withdrawCash(50)


    Withdrew $50.00 from your portfolio.
    You have $166.45 remaining in your portfolio.
        


In [26]:
portfolio.withdrawCash(500000)


    Insufficient funds.
    You have $166.45 remaining in your portfolio.
        


In [27]:
portfolio.history()

03:39PM on 08/08/2018 | Added cash: $300.50
03:39PM on 08/08/2018 | Bought shares: 5.00 (HFH) @ $20.00/s ($100.00 total)
03:39PM on 08/08/2018 | Bought shares: 10.30 (BRT) @ $1.00/s ($10.30 total)
03:39PM on 08/08/2018 | Bought shares: 2.00 (GHT) @ $1.00/s ($2.00 total)
03:39PM on 08/08/2018 | Sold shares: 3.00 (BRT) for $1.14/s ($3.42 total)
03:39PM on 08/08/2018 | Sold shares: 1.00 (HFH) for $24.83/s ($24.83 total)
03:39PM on 08/08/2018 | Withdrew cash: $50.00


In [28]:
portfolio.history(from_start = False)

03:39PM on 08/08/2018 | Withdrew cash: $50.00
03:39PM on 08/08/2018 | Sold shares: 1.00 (HFH) for $24.83/s ($24.83 total)
03:39PM on 08/08/2018 | Sold shares: 3.00 (BRT) for $1.14/s ($3.42 total)
03:39PM on 08/08/2018 | Bought shares: 2.00 (GHT) @ $1.00/s ($2.00 total)
03:39PM on 08/08/2018 | Bought shares: 10.30 (BRT) @ $1.00/s ($10.30 total)
03:39PM on 08/08/2018 | Bought shares: 5.00 (HFH) @ $20.00/s ($100.00 total)
03:39PM on 08/08/2018 | Added cash: $300.50
